# Clasificador Multietiqueta de Eventos Sísmicos
Este *notebook* procesa una **base de acelerogramas** y genera un conjunto de datos limpio y uniformemente muestreado para modelos de *Machine Learning*.  
Los bloques de código originales **no se alteran** en su lógica; solo se añaden comentarios detallados y celdas *Markdown* para guiar la lectura.

## Bloque 1 – Importación de librerías

In [26]:
# ----------------------------------------------------------------------------------
# BLOQUE 1: Importación de librerías
# ----------------------------------------------------------------------------------
import os
import re
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from itertools import groupby, zip_longest
import pickle

## Bloque 2 – Extracción de datos brutos

In [27]:
# ----------------------------------------------------------------------------------
# BLOQUE 2: Preparación del directorio y descompresión de archivos
# ----------------------------------------------------------------------------------
# Si el directorio no es Seismic-Multilabel-Event-Classifier, se sale un directorio
if not os.path.basename(os.getcwd()) == 'Seismic-Multilabel-Event-Classifier':
    os.chdir('..')
    print(f"Changed directory to {os.getcwd()}")
import zipfile
zip_path = 'data/raw/Base-sismos-2024.zip'
extract_to = 'data/raw/Base-sismos-2024'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
print(f"Extracted {zip_path} to {extract_to}")

Extracted data/raw/Base-sismos-2024.zip to data/raw/Base-sismos-2024


## Bloque 3 – Función `processNGAfile`

In [ ]:
# ----------------------------------------------------------------------------------
# BLOQUE 3: Definición de la función `processNGAfile`
# ----------------------------------------------------------------------------------
"""
@author: Daniel Hutabarat - UC Berkeley, 2017
"""

def processNGAfile(filepath, scalefactor=None):
    '''
    Esta función procesa un historial de aceleración de un archivo de datos NGA (.AT2)
    a un vector de una sola columna y devuelve el número total de puntos de datos y
    el intervalo de tiempo de la grabación.

    Parámetros:
    ------------
    filepath : string
        Ruta y nombre del archivo.
    scalefactor : float (opcional)
        Factor de escala que se aplica a cada componente del arreglo de aceleración.

    Salida:
    ------------
    npts: número total de puntos registrados (datos de aceleración)
    dt: intervalo de tiempo entre los puntos registrados
    time: array (n x 1) - arreglo de tiempos, misma longitud que npts
    inp_acc: array (n x 1) - arreglo de aceleraciones, misma longitud que time,
             la unidad usualmente es en g (gravedad) a menos que se indique lo contrario.

    Ejemplo (graficar tiempo vs aceleración):
    filepath = os.path.join(os.getcwd(),'motion_1')
    npts, dt, time, inp_acc = processNGAfile(filepath)
    plt.plot(time, inp_acc)
    '''
    try:
        if not scalefactor:
            scalefactor = 1.0  # Si no se proporciona, usa un factor de escala por defecto de 1.0

        with open(filepath, 'r') as f:
            content = f.readlines()  # Lee todas las líneas del archivo

        counter = 0
        desc, row4Val, acc_data = "", "", []

        for x in content:
            if counter == 3:
                # En la línea 4 se suele encontrar la información NPTS y DT
                row4Val = x
                if row4Val[0][0] == 'N':
                    npts_match = re.search(r'NPTS\s*=\s*([0-9.]+)', row4Val)
                if npts_match:
                    npts = float(npts_match.group(1))  # Número total de puntos
                else:
                    raise ValueError("No se encontró un valor para NPTS.")
                dt_match = re.search(r'DT\s*=\s*([0-9.]+)', row4Val)
                if dt_match:
                    dt = float(dt_match.group(1))  # Intervalo de tiempo entre puntos
                else:
                    raise ValueError("No se encontró un valor para DT.")
            elif counter == 4:
                # En la línea 5 puede comenzar la data, si no hay encabezados adicionales
                row4Val = x
                #print(row4Val)
                # Si comienza directamente con números o signos, asume que son datos de aceleración
                if row4Val[0][0] == '.' or row4Val[0][0] == '-' or row4Val[0][0].isdigit() or row4Val[0][0] == ' ':
                    print("Datos de aceleración encontrados en la línea 5.")
                    data = str(x).split()
                    for value in data:
                        a = float(value) * scalefactor
                        acc_data.append(a)

                    # Convierte lista a array de numpy
                    inp_acc = np.asarray(acc_data)

                    # Crea el vector de tiempo con base en el número de puntos y el dt
                    time = []
                    for i in range(len(acc_data)):
                        t = i * dt
                        time.append(t)

            elif counter > 4:
                # Las siguientes líneas después de la 5ta contienen más datos de aceleración
                data = str(x).split()
                for value in data:
                    a = float(value) * scalefactor
                    acc_data.append(a)

                inp_acc = np.asarray(acc_data)

                # Genera de nuevo el vector de tiempo basado en la longitud actual
                time = []
                for i in range(len(acc_data)):
                    t = i * dt
                    time.append(t)

            counter = counter + 1

        # Devuelve los resultados procesados
        return npts, dt, time, inp_acc

    except IOError:
        print("¡processMotion FALLÓ!: El archivo no se encuentra en el directorio.")


## Bloque 4 – Función `procesarDatos`

In [ ]:
# ----------------------------------------------------------------------------------
# BLOQUE 4: Definición de la función `procesarDatos`
# ----------------------------------------------------------------------------------
def procesarDatos(ruta_archivo):
    # Lista general que almacenará los registros de todas las carpetas procesadas
    registro = []

    # Subcarpetas específicas que contienen los archivos que queremos procesar
    subcarpetas_especificas = [
        "Componente Horizontal 1",
        "Componente Horizontal 2",
        "Componente Vertical",
    ]

    # Recorre recursivamente todas las carpetas, subcarpetas y archivos desde la ruta base
    for carpeta_raiz, subcarpetas, archivos in os.walk(ruta_archivo):
        registro1 = []  # Acumula los registros de una carpeta raíz específica

        # Revisa cada subcarpeta para verificar si es una de las que nos interesa
        for subcarpeta in subcarpetas:
            if subcarpeta in subcarpetas_especificas:
                ruta_subcarpeta = os.path.join(carpeta_raiz, subcarpeta)
                print(f"Procesando subcarpeta: {ruta_subcarpeta}")

                registros = []  # Lista temporal de registros para esta subcarpeta

                # Procesa todos los archivos en la subcarpeta
                for archivo in os.listdir(ruta_subcarpeta):
                    # Asegura que sea un archivo (y no una carpeta)
                    if os.path.isfile(os.path.join(ruta_subcarpeta, archivo)):
                        ruta_archivo = os.path.join(ruta_subcarpeta, archivo)
                        #print(f"Procesando archivo: {ruta_archivo}")

                        # Usa la función externa processNGAfile para extraer datos del archivo
                        ntps, dt, time, inp_acc = processNGAfile(ruta_archivo)

                        # Extrae metadatos a partir de la estructura del path del archivo
                        rutaS = ruta_archivo.split('/')
                        falla = rutaS[rutaS.index('Base-sismos-2024') + 1]  # Nombre de la falla
                        mag = re.search(r'(\d+-\d+)', rutaS[rutaS.index('Base-sismos-2024') + 2]).group(0)  # Rango de magnitud
                        vs = rutaS[rutaS.index('Base-sismos-2024') + 3].split("Vs30.")[1].strip()  # Valor de Vs30
                        tipo = rutaS[rutaS.index('Base-sismos-2024') + 4]  # Tipo de aceleración (H1, H2, V)
                        # Crea un diccionario con todos los datos y lo añade a la lista
                        registros.append({
                            'Archivo': archivo,
                            'NPTS': ntps,
                            'DT': dt,
                            'Falla': falla,
                            'Mag': mag,
                            'Vs': vs,
                            'Time': time,
                            'Acc': inp_acc,
                            'Tipo': tipo
                        })

                # Agrega todos los registros procesados de esta subcarpeta al conjunto de esta carpeta raíz
                registro1.extend(registros)

        # Si se procesaron registros en esta carpeta raíz, se agregan al registro global
        if len(registro1) != 0:
            print("Registro de una carpeta completado")
            registro.append(registro1)

    # Devuelve todos los registros organizados por carpeta raíz
    return registro



## Bloque 5 – Procesamiento inicial

In [ ]:
# ----------------------------------------------------------------------------------
# BLOQUE 5: Ejecución de `procesarDatos`
# ----------------------------------------------------------------------------------
ruta_base = 'data/raw/Base-sismos-2024'
datos_procesados = procesarDatos(ruta_base)
print("Datos procesados.")

## Bloque 6 – Serialización

In [ ]:
# ----------------------------------------------------------------------------------
# BLOQUE 6: Guardar resultados con `pickle`
# ----------------------------------------------------------------------------------
os.makedirs('data/interim', exist_ok=True)
with open('data/interim/datosML.pkl', 'wb') as f:
    pickle.dump(datos_procesados, f)

## Bloque 7 – Deserialización

In [28]:
# ----------------------------------------------------------------------------------
# BLOQUE 7: Cargar datos serializados
# ----------------------------------------------------------------------------------
with open('data/interim/datosML.pkl', 'rb') as f:
    registros_por_carpeta = pickle.load(f)
print("Datos cargados.")

Datos cargados.


## Bloque 8 – Agrupación y limpieza

In [29]:
# ----------------------------------------------------------------------------------
# BLOQUE 8: Agrupar componentes, validar y limpiar duplicados
# ----------------------------------------------------------------------------------
from collections import defaultdict
resultados = []
for archivos in registros_por_carpeta:
    grupos = defaultdict(list)
    for archivo in archivos:
        nombre_base = archivo['Archivo'].rsplit('_', 1)[0]
        grupos[nombre_base].append(archivo)

    for nombre_base, archivos_grupo in grupos.items():
        npts = archivos_grupo[0]['NPTS']
        dt = archivos_grupo[0]['DT']
        if len(archivos_grupo) != 3:
            continue
        # Ajuste de NPTS discrepantes
        if any(archivo['NPTS'] != npts for archivo in archivos_grupo):
            minNpts = int(min(a['NPTS'] for a in archivos_grupo))
            for archivo in archivos_grupo:
                archivo['NPTS'] = minNpts
                archivo['Time'] = archivo['Time'][:minNpts]
                archivo['Acc'] = archivo['Acc'][:minNpts]

        agrupado = {
            'Archivo': nombre_base,
            'NPTS': npts,
            'DT': dt,
            'Falla': archivos_grupo[0]['Falla'],
            'Mag': archivos_grupo[0]['Mag'],
            'Vs': archivos_grupo[0]['Vs'],
            'Time': archivos_grupo[0]['Time'],
            'AccV': None,
            'AccH2': None,
            'AccH1': None
        }
        for archivo in archivos_grupo:
            if archivo['Tipo'] == 'Componente Horizontal 1':
                agrupado['AccH1'] = archivo['Acc']
            elif archivo['Tipo'] == 'Componente Horizontal 2':
                agrupado['AccH2'] = archivo['Acc']
            elif archivo['Tipo'] == 'Componente Vertical':
                agrupado['AccV'] = archivo['Acc']
        resultados.append(agrupado)
print(f"Total registros agrupados: {len(resultados)}")

Total registros agrupados: 1696


## Bloque 9 – Normalización temporal

In [30]:
# ----------------------------------------------------------------------------------
# BLOQUE 9: Función `normalizarSeries`
# ----------------------------------------------------------------------------------
from scipy.interpolate import interp1d
def normalizarSeries(grupos):
    dt = 0.01
    for g in grupos:
        t_orig = g['Time']
        t_new = np.arange(0, t_orig[-1] + dt, dt)
        g['AccH1'] = interp1d(t_orig, g['AccH1'], kind='linear', bounds_error=False, fill_value='extrapolate')(t_new)
        g['AccH2'] = interp1d(t_orig, g['AccH2'], kind='linear', bounds_error=False, fill_value='extrapolate')(t_new)
        g['AccV']  = interp1d(t_orig, g['AccV'],  kind='linear', bounds_error=False, fill_value='extrapolate')(t_new)
        g['Time'] = t_new
    return grupos

## Bloque 10 – Aplicar normalización

In [31]:
# ----------------------------------------------------------------------------------
# BLOQUE 10: Aplicar `normalizarSeries`
# ----------------------------------------------------------------------------------
grupos_normalizados = normalizarSeries(resultados)
print("Normalización completada.")

Normalización completada.


## Bloque 11 – DataFrame

In [32]:
# ----------------------------------------------------------------------------------
# BLOQUE 11: Conversión a DataFrame y limpieza
# ----------------------------------------------------------------------------------
df = pd.DataFrame(grupos_normalizados)
df = df.drop(columns=['DT'])  # eliminar DT redundante
df.head()

,Archivo,NPTS,Falla,Mag,Vs,Time,AccV,AccH2,AccH1
0,RSN8478_PARK2004,32169.0,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[3.3443977e-08, 3.3695554e-08, 3.3943394e-08, ...","[-5.5774385e-08, -5.6530193e-08, -5.7289814e-0...","[-1.931981e-08, -1.9326151e-08, -1.9334876e-08..."
1,RSN8700_40204628,20001.0,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[8.5424888e-09, 8.8524636e-09, 9.1842936e-09, ...","[-2.2300064e-09, -2.3446809e-09, -2.4440089e-0...","[8.4741547e-10, 1.2447632e-09, 1.6283558e-09, ..."
2,RSN8459_PARK2004,32380.0,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[5.855578e-08, 5.9356899e-08, 6.0156329e-08, 6...","[2.7169869e-08, 2.7004326e-08, 2.6835684e-08, ...","[1.5000493e-08, 1.4939016e-08, 1.4878223e-08, ..."
3,RSN2148_BEARCTY,8200.0,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[-2.276836e-05, -2.118971e-05, -2.810751e-05, ...","[7.43878e-06, 5.521958e-06, -2.429367e-06, -6....","[1.001939e-05, 9.63514e-06, 1.253978e-05, 1.00..."
4,RSN8426_BEARCTY,14465.0,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[3.079444e-09, 3.07869528e-09, 3.07794452e-09,...","[4.3128452e-10, 4.27025392e-10, 4.22775078e-10...","[2.9098055e-09, 2.89089174e-09, 2.87200576e-09..."


## Bloque 12 – Guardar JSON

In [33]:
# ----------------------------------------------------------------------------------
# BLOQUE 12: Guardar DataFrame a JSON
# ----------------------------------------------------------------------------------
df.to_json('data/interim/datosML.json', orient='records', lines=True)